In [36]:
from importlib import reload
import qubic.job_manager_jpm as _jm
import chipcalibration.ramsey_jpm as _ramsey
from matplotlib import pyplot as plt
from distproc.hwconfig import FPGAConfig, load_channel_configs
import chipcalibration.config as _cfg
import qubic.rpc_client as _rc
import numpy as np
import pygsti
from pygsti.models import modelconstruction as _mc

In [37]:
reload(_jm)
reload(_ramsey)

<module 'chipcalibration.ramsey_jpm' from '/home/jpmarceaux/PycharmProjects/chipcalibration/chipcalibration/ramsey_jpm.py'>

In [38]:
fpga_config = FPGAConfig(**{'fpga_clk_period': 2.e-9, 'alu_instr_clks': 5, 'jump_cond_clks': 5, 'jump_fproc_clks': 5,
                            'pulse_regwrite_clks': 3})

qchipname = 'X4Y2'
fpga_config, qchip, channel_config = _cfg.load_configs(qchipname, qchip_file='qubitcfg.json')



In [39]:
# setup a qubic jobmanager
runner = _rc.CircuitRunnerClient(ip='192.168.1.247')
jm = _jm.JobManager(fpga_config, channel_config, runner)

In [46]:
target_qid = 'Q2'
readout_register = ['Q1', 'Q2']
delay_interval = np.linspace(0,50.e-6,75)
initial_drive_frequency = qchip.qubits['Q2'].freq
print('Initial drive frequency:', initial_drive_frequency)

Initial drive frequency: 4943277065.23312


In [47]:
# setup a simulation manager
# make the processor spec
register = ['Q1']  # these correspond to the actual qubit id's on the qubic device
gate_names = ['Gxpi2', 'Gzpi2']

availability = {
    'Gxpi2': [(qid,) for qid in register],
    'Gzpi2': [(qid,) for qid in register],
}

pspec = pygsti.processors.QubitProcessorSpec(num_qubits=len(register), gate_names=gate_names,
                                             availability=availability, qubit_labels=register)

# make a pygsti simulation manager
model = _mc.create_explicit_model(pspec)
model.set_all_parameterizations('TP')
model = model.depolarize(0.01)
sim_manager = _jm.SimulationManager(model)


In [49]:
iramsey = _ramsey.RamseyExperiment([target_qid], readout_register, delay_interval, initial_drive_frequency)

In [50]:
iramsey.run_ramsey(initial_drive_frequency, delay_interval, sim_manager, 1000, qchip=None)

TypeError: SimulationManager.collect_classified_shots() got multiple values for argument 'reads_per_shot'

In [ ]:
# TODO setup the simulation manager to take midcircuit measurements